In [1]:
from __future__ import print_function
import tensorflow as tf
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Activation, Convolution2D, GlobalAveragePooling2D, merge
from keras.utils import np_utils
from keras.optimizers import SGD
from keras import backend as K
from keras.models import Model
from keras.layers.core import Lambda
from keras.callbacks import ModelCheckpoint
import pandas
import cv2
import numpy as np

K.set_image_dim_ordering('tf')


batch_size = 32
nb_classes = 10
nb_epoch = 350


rows, cols = 32, 32

channels = 3

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

print (X_train.shape[1:])

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

model = Sequential()

model.add(Convolution2D(96, 3, 3, border_mode = 'same', input_shape=(32, 32, 3)))
model.add(Activation('relu'))
model.add(Convolution2D(96, 3, 3,border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(96, 3, 3, border_mode='same', subsample = (2,2)))
model.add(Dropout(0.5))

model.add(Convolution2D(192, 3, 3, border_mode = 'same'))
model.add(Activation('relu'))
model.add(Convolution2D(192, 3, 3,border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(192, 3, 3,border_mode='same', subsample = (2,2)))
model.add(Dropout(0.5))

model.add(Convolution2D(256, 3, 3, border_mode = 'same'))
model.add(Activation('relu'))
model.add(Convolution2D(256, 1, 1,border_mode='valid'))
model.add(Activation('relu'))
model.add(Convolution2D(10, 1, 1, border_mode='valid'))



model.add(GlobalAveragePooling2D())
model.add(Activation('softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

print (model.summary())

model_json = model.to_json()
with open("allcnn_model.json", "w") as json_file:
    json_file.write(model_json)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True) 

datagen.fit(X_train)
filepath="weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='max')

callbacks_list = [checkpoint]
    # Fit the model on the batches generated by datagen.flow().
history_callback = model.fit_generator(datagen.flow(X_train, Y_train,
                        batch_size=batch_size),
                        samples_per_epoch=X_train.shape[0],
                        nb_epoch=nb_epoch,
                        validation_data=(X_test, Y_test),
                        callbacks=callbacks_list, verbose=1)


Using TensorFlow backend.
W0806 20:38:17.558586 140461137114944 deprecation_wrapper.py:119] From /opt/app-root/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0806 20:38:17.566881 140461137114944 deprecation_wrapper.py:119] From /opt/app-root/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0806 20:38:17.568415 140461137114944 deprecation_wrapper.py:119] From /opt/app-root/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0806 20:38:17.591195 140461137114944 deprecation_wrapper.py:119] From /opt/app-root/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_de

X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
(32, 32, 3)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 96)        2688      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 96)        83040     
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 96)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 96)        83040     
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 96)        0         
_________________________________________________________

W0806 20:38:18.312785 140461137114944 deprecation.py:323] From /opt/app-root/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/350
1562/1562 [==============================] - 54s 34ms/step - loss: 2.0547 - acc: 0.2120 - val_loss: 1.9229 - val_acc: 0.2612

Epoch 00001: val_acc improved from -inf to 0.26120, saving model to weights.hdf5
Epoch 2/350
1562/1562 [==============================] - 52s 33ms/step - loss: 1.7197 - acc: 0.3371 - val_loss: 1.6132 - val_acc: 0.3888

Epoch 00002: val_acc improved from 0.26120 to 0.38880, saving model to weights.hdf5
Epoch 3/350
1562/1562 [==============================] - 52s 33ms/step - loss: 1.5305 - acc: 0.4307 - val_loss: 1.3689 - val_acc: 0.5038

Epoch 00003: val_acc improved from 0.38880 to 0.50380, saving model to weights.hdf5
Epoch 4/350
1562/1562 [==============================] - 52s 33ms/step - loss: 1.3761 - acc: 0.4981 - val_loss: 1.4030 - val_acc: 0.5297

Epoch 00004: val_acc improved from 0.50380 to 0.52970, saving model to weights.hdf5
Epoch 5/350
1562/1562 [==============================] - 52s 33ms/step - loss: 1.2487 - acc: 0.5504 - val_loss: 1.0

KeyboardInterrupt: 

In [ ]:
# im = cv2.resize(cv2.imread('image.jpg'), (224, 224)).astype(np.float32)
# out = model.predict(im)
# print np.argmax(out)

# pandas.DataFrame(history_callback.history).to_csv("history.csv")